# 神经网络：感知机Perceptron

感知机Perceptron是一种**人工神经元**。   
在现代的神经网络工作中，更常用的是另一种人工神经元--**sigmoid神经元**。

## Udacity：
![title](../images/perceptron_u.png)

图解：   
激活函数activation function： $${\sum_{i=1}^k}x_i \cdot w_i$$
阈值firing threshold：$\theta$
符号函数sign function：$$sign(x) = \begin{cases} +1, & x \ge 0 \\ 0, & x < 0 \end{cases}$$

在图中的例子$x_1 = 1, x_2 = 0, x_3 = -1.5$，$w_1 = 1/2, w_2 = 3/5, w_3 = 1$，
求得的结果为-1，-1小于0，所以$y = 0$。

#### 感知器得出的都是线性关系，所有的感知器都将计算线性问题，始终都将计算半平面halfplanes。
感知器就是一个线性函数，它计算的是超平面。

NAND门（与非门）可以用来做基本逻辑运算。   
事实上，我们可以使用感知机网络做任何逻辑运算。   
因为**与非门**是通用的计算，我们可以用与非门实现任何运算。

<a href = 'https://www.jianshu.com/p/921e9c6be305'>与非门示例</a>

### Perceptron Trainging:
Given examples, find weights that map inputs to outputs.
- perceptron rule   (threshold)
- gradient descent / delta rule ( unthreshold)

感知机训练法则在**样本线性可分**时，能够找到最佳的权重向量，使得所有样本均被正确分类。    
如果**样本线性不可分**，则不能收敛。    

不能收敛，也无法找到一个权向量使得分类错误率最小吗？   

而采用梯度下降法，能够在训练样本线性不可分的情况下，找到目标的最佳近似，即分类错误率最小。   

#### 感知机训练法则使得线性可分的样本集得到完全正确的分类（完美拟合）。
#### 梯度下降法，在线性可分时能够达到一定的效果（全部正确分类），而且在线性不可分的时候，能够找到使分类错误率最小的最佳拟合。

<a href='https://blog.csdn.net/bingduanlbd/article/details/24468885'>参考博文</a>

## perceptron rule (threshold)
![title](../images/Perceptron_rule.png)



## gradient descent / delta rule ( unthreshold)
![title](../images/Gradient_descent.png)

#### 上图理解：
- 第一张图，Perceptron Rule，我们仅针对$y-\hat y$去的到错误的结果数，只考虑了错误数对参数的影响。
- 第二张图通过求导解释了$L2$范数$\frac{1}{2}$对结果的影响。

两个例子的区别：
感知机规则的例子中进行了阈值化，而在另一个例子梯度下降中未进行阈值化。

In [2]:
import numpy as np 

class Perceptron1(object):
    '''
    根据Perceptron Rule训练感知机模型
    '''
    def __init__(self):
        self.w = None
        self.b = None
    
    def _recursion(x, diffs):
        for idx, d in enumerate(list(diffs)):
                delta = lr * d * x[idx]
                self.w[idx] += delta
        
    def fit(self, x, y, lr=0.01, epoch=1):
        x = np.array(x)
        y = np.array(y)
       
        self.w = np.zeros(x.shape[1])
        
        self.b = 0
        
        for _ in range(epoch):
            
            y_pred = np.dot(x, self.w) + self.b
            for idx, e in enumerate(y_pred):
                if e >=0:
                    y_pred[idx] = 1
                else:
                    y_pred[idx] = -1
               
            diffs = y - y_pred
            diffs = diffs[:, 0]
        
            # print(diffs.sum())
           
            if diffs.sum() == 0:
                print('END：', self.w, self.b)
            else:
                
                for idx, d in enumerate(list(diffs)):
                    # 单个的更新有问题
                    delta = lr * d * x[idx]
                    print(delta)
                    self.w += delta
                    self.b += lr*d

            
perceptron_obj = Perceptron1()
X_train = [[1,1], [2,2], [3,3], [4,4], [-1,-1]]
y_train = [[0], [0], [1], [1], [0]]
perceptron_obj.fit(X_train, y_train)   

[-0.01 -0.01]
[-0.02 -0.02]
[0. 0.]
[0. 0.]
[0.01 0.01]


In [3]:
class Perceptron(object):
    """Perceptron classifier.
    Parameters
    ------------
    eta:float,Learning rate (between 0.0 and 1.0)
    n_iter:int,Passes over the training dataset.
     
    Attributes
    -------------
    w_: 1d-array,Weights after fitting.
    errors_: list,Numebr of misclassifications in every epoch.
    """
    def __init__(self,eta=0.01,n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
        
    def fit(self,X,y):
        """Fit training data.先对权重参数初始化，然后对训练集中每一个样本循环，根据感知机算法学习规则对权重进行更新
        Parameters
        ------------
        X: {array-like}, shape=[n_samples, n_features]
            Training vectors, where n_samples is the number of samples and n_featuers is the number of features.
        y: array-like, shape=[n_smaples]
            Target values.
        Returns
        ----------
        self: object
        """
        self.w_ = np.zeros(1 + X.shape[1]) # add w_0　　　　　#初始化权重。数据集特征维数+1。
        self.errors_ = []#用于记录每一轮中误分类的样本数
         
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X,y):
                update = self.eta * (target - self.predict(xi))#调用了predict()函数
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
     
    def net_input(self,X):
        """calculate net input"""
        return np.dot(X,self.w_[1:]) + self.w_[0]#计算向量点乘
    
    def predict(self,X):#预测类别标记
        """return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0,1,-1)

perceptron_obj = Perceptron()
X_train = np.array([[1,1], [2,2], [3,3], [4,4], [-1,-1]])
y_train = np.array([[0], [0], [1], [1], [0]])
model = perceptron_obj.fit(X_train, y_train) 
print(model.w_)

[ 2.00000000e-02 -1.56125113e-17 -1.56125113e-17]


**梯度下降法**：
- 第一步：求损失函数的梯度（求导）
- 第二步：梯度是函数值增长最快的方向，我们想要最小化损失函数，想要让函数值减小得最快，就是将参数沿着梯度的反方向走一步。

In [7]:
print('+'*100)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


## 李航：
感知机（perceptron）是二类分类的线性分类模型，其输入为实例的特征向量，输出为实例的类别，取 +1 和 -1 二值。   
感知机对应于输入空间（特征空间）中将实例划分为正负两类的分离超平面，属于判别模型。   
感知机学习旨在求出将训练数据进行线性划分的分离超平面，为此，导入基于误分类的损失函数，利用梯度下降法对损失函数进行极小化，求得感知机模型。   
感知机学习算法具有简单而易于实现的优点，分为原始形式和对偶形式。    
感知机预测是用学习得到的感知机模型对新的输入实例进行分类。  
感知机1957年由Rosenblatt提出，是神经网络与支持向量机的基础。 

### 感知机模型
感知机定义：   
假设输入空间（特征空间）$X \subseteq R^{n}$，输出空间是$y =\{+1, -1\}$。输入$x\in{X}$表示实例的特征向量，对应于输入空间（特征空间）的点；输出$y\in Y$表示实例的类别。   
由输入空间到输出空间的函数：
$$f(x) = sign(w\cdot x +b)$$ 称为感知机。   

其中，$w$和$b$为感知机模型参数，$w\in R^{n}$叫做权值（weight）或权值向量(weight vector)，$b\in R$叫做偏置(bias)，     
$w \cdot x$表示$w$和$x$的内积。    
sign是符号函数，即$$sign(x) = \begin{cases} +1, & x \ge 0 \\ -1, & x < 0 \end{cases}$$


感知机是一种线性分类模型，属于判别模型。   
感知机模型的假设空间是定义在特征空间中的所有线性分类模型（linear classification model）或线性分类器（linear classifier），即函数集合$\{ f | f(x) = w \cdot x + b\}$。


感知机的几何解释：  
线性方程：$$w \cdot x + b = 0$$
对应于特征空间$R^{n}$中的一个超平面$S$，其中$w$是超平面的法向量，$b$是超平面的截距。 这个超平面将特征空间划分为两个部分。   
位于两部分的点（特征向量）分别被分为正、负两类。   
因此，超平面$S$称为分离超平面(separating hyperplane)。

![title](../images/perceptron_model1.png)


感知机学习，由训练数据集（实例的特征向量及类别）  
$$ T = \{(x_1，y_1), (x_2, y_2), ... , (x_N, y_N)\}$$   
其中，$x_i \in X = R^n$，$y_i \in Y = \{+1, -1\}$，$i = 1,2,...,N$，求得感知机模型，即求得模型参数$w$，$b$。   
感知机预测，通过学习得到的感知机模型，对于新年的输入实例给出其对应的输出类别。

### 感知机的学习策略（损失函数）
#### 1.数据集的线性可分性：
给定一个数据集$$T=\{(x_1, y_1), (x_2, y_2), ... , (x_N, y_N)\}$$
其中，$x_i \in X = R^n$, $y_i in Y = \{+1, -1\}$, $i = 1,2,...,N$,  如果存在某个超平面$S$
$$w \cdot x + b = 0$$
能够将数据集的正实例点和负实例点完全正确地划分到超平面的两侧，即对所有$y_i = +1 $的实例$i$，有$w \cdot x_i + b > 0$，对所有$y_i = -1$的实例$i$，有$w \cdot x_i + b < 0$，则称数据集$T$为线性可分数据集（linear separable data set）；否则，称数据集$T$线性不可分。


#### 2.感知机学习策略
假设训练数据集是线性可分的，感知机学习的目标是求得一个能够将训练集正实例点和负实例点完全正确分开的分离超平面。    
为了找出这样的超平面，即确定感知机模型参数$w$，$b$，需要确定一个学习策略，即**定义（经验）损失函数并将损失函数最小化。**      

**损失函数：**
- 损失函数的一个自然选择是误分类点的总数。   
    - 但是，这样的损失函数不是参数$w$，$b$的连续可导函数，不易优化。    
- 损失函数的另一个选择是误分类点到超平面$S$的总距离，这是感知机所采用的。
    - 为此，首先写出输入空间$R^n$中任一点$x_0$到超平面$S$的距离：
        $$\frac{1}{||w||}|w \cdot x_0 + b |$$
        这里，$||w||$是$w$的$L_2$范数。
    - 其次，对于误分类的数据$(x_i, y_i)$来说：
        $$-y_i(w \cdot x_i + b ) > 0$$
    - 这样，假设超平面$S$的误分类点的集合为$M$，那么所有误分类点到超平面$S$的总距离为
        $$- \frac{1}{||w||} \sum_{x_i \in M} y_i(w \cdot x_i + b)$$
    - 不考虑$- \frac{1}{||w||}$，就得到感知机学习的损失函数。
    
由上可得，给定训练数据集
$$ T = \{(x_1, y_1), (x_2, y_2), ... , (x_N, y_N)\}$$
其中，$x_i \in X = R^n$，$y_i \in Y = \{ +1, -1\}$，$i = 1,2,...,N$。感知机$sign(w \cdot x + b)$学习的损失函数定义为
$$L(w, b) = -\sum_{x_i \in M} y_i(w \cdot x_i + b)$$
其中$M$为误分类点的集合。   
这个损失函数就是感知机学习的经验风险函数。    

显然，损失函数$L(w,b)$是非负的。    
如果没有误分类点，损失函数值是0。   
而且，误分类点越少，误分类点离超平面越近，损失函数值就越小。   

一个特定的样本点的损失函数：   
在误分类时是参数$w$，$b$的线性函数，在正确分类时是0。   
因此，给定训练数据集$T$，损失函数$L(w,b)$是$w$，$b$的连续可导函数。   

感知机学习的策略是在假设空间中选取使损失函数$$L(w, b) = -\sum_{x_i \in M} y_i(w \cdot x_i + b)$$最小的模型参数$w$，$b$，即感知机模型。

### 感知机学习算法：原始形式、对偶形式
感知机学习问题转化为求解损失函数$$L(w, b) = -\sum_{x_i \in M} y_i(w \cdot x_i + b)$$的最优化问题，最优化的方法是随着梯度下降法。        

关于感知机学习的具体算法：
- 原始形式
- 对偶形式


### 1.感知机学习算法的原始形式
感知机学习算法是对以下最优化问题的算法。   
给定一个训练数据集
$$ T = \{(x_1, y_1), (x_2, y_2), ... , (x_N, y_N)\}$$
其中，$x_i \in X = R^n$，$y_i \in Y = \{ +1, -1\}$，$i = 1,2,...,N$。
求参数$w$，$b$，使其为以下损失函数极小化问题的解
$$\min_{w,b} L(w, b) = -\sum_{x_i \in M} y_i(w \cdot x_i + b)$$
其中，$M$为误分类点的集合。  

感知机学习算法时误分类驱动的，具体采用随机梯度下降法（stochastic gradient descent）。    
首先，任意选取一个超平面$w_0$，$b_0$，然后用梯度下降法不断地极小化目标函数$$\min_{w,b} L(w, b) = -\sum_{x_i \in M} y_i(w \cdot x_i + b)$$
极小化过程中不是一次使$M$中所有误分类点的梯度下降，而是一次随机选取一个误分类点使其梯度下降。   

假设误分类点集合$M$是固定的，那么损失函数$L(w,b)$的梯度由
$$\nabla_w L(w,b) = - \sum_{x_i \in M} y_i x_i$$
$$\nabla_b L(w,b) = - \sum_{x_i \in M} y_i$$
给出。


随机选取一个误分类点$(x_i, y_i)$，对$w$，$b$进行更新：
$$w \leftarrow w + \eta y_i x_i$$
$$b \leftarrow b + \eta y_i$$
式中$\eta( 0 < \eta \leq 1）$是步长，在统计学习中又称学习率（learning rate）。   
这样，通过迭代可以期待损失函数$L(w,b)$不断减小，直到为0。     


<p style='color:blue;font-size:22px;'>算法：感知机学习算法的原始形式</p>

**输入：**训练数据集$T = \{(x_1, y_1), (x_2, y_2), ... , (x_N, y_N)\}$，其中，$x_i \in X = R^n$，$y_i \in Y = \{ +1, -1\}$，$i = 1,2,...,N$，学习率$\eta( 0 < \eta \leq 1$；   
**输出：**$w$，$b$；感知机模型$f(x) = sign(w \cdot x +b)$     
- 选取初值$w_0$，$b_0$；
- 在训练集中选取数据$(x_i, y_i)$
- 如果$y_i (w \cdot x_i + b) \leq 0 $
$$w \leftarrow w + \eta y_i x_i$$
$$b \leftarrow b + \eta y_i$$
- 回到第二步，直至训练集中没有误分类点。  

这种学习算法直观上有如下解释：   
当一个实例点被误分类，即位于分离超平面的错误一侧时，则调整$w$，$b$的值，使分离超平面向该误分类点的一侧移动，以减少该误分类点与超平面间的距离，直至超平面越过该误分类点使其被正确分类。    



<p style='color:red'>假设误分类点集合$M$是固定的，那么损失函数$L(w,b)$的梯度由
$$\nabla_w L(w,b) = - \sum_{x_i \in M} y_i x_i$$
$$\nabla_b L(w,b) = - \sum_{x_i \in M} y_i$$
给出
++++++++ 此处不理解，为啥梯度是这么算出来的！！！微积分快点补！！！！大笨蛋！！！！</p>


### 算法的收敛性
证明见书籍，目前微积分未补的情况下，难理解。

对偶形式的算法在SVM期间对应了一起看。